In [1]:
%%html
<link rel="stylesheet" href="static/hyrule.css" type="text/css">

## Matrix Algebra and the Regression Algorithm

### Objectives

* Understand how Matrix Algebra is used to find coefficients in an ordinary least squares regression
* Review two techniques to improving the linear model
* Understanding the use case and results of a Logistic Regression
* New Cross Validation Technique - K-Fold
* First Metric for Classification: Accuracy

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm

practice_set = pd.DataFrame({
    'x': [3.385, 0.48, 1.35, 465, 36.33],
    'y': [44.5, 15.5, 8.1, 423, 119.5],
})

A = np.array([[1, 1, 1, 1, 1], practice_set['x']])
# print np.linalg.inv(A.dot(A.T)).dot(A.dot(practice_set['y']))

model = lm.LinearRegression().fit(practice_set[['x']], practice_set['y'])
#print model.intercept_
#print model.coef_

In [3]:
model.intercept_

37.200896079333063

In [4]:
model.coef_

array([ 0.83821876])

In [5]:
### Statsmodels, because we love statsmodels,
### and because logistic regressions are still common for data exploration:from sklearn import datasets.
### Since logistic is used to solve binomial problems, our targets should be 0 and 1.
### by default, fit_regularized uses L1 Regularization
from sklearn import datasets
import numpy as np
import statsmodels.formula.api as smf
iris = datasets.load_iris()
irisdf = pd.DataFrame(iris.data, columns=['sep_len', 'sep_wid', 'pet_len', 'pet_wid'])
irisdf['target'] = iris.target
irisdf = irisdf.query('target in (1, 2)')
irisdf['target'] = irisdf['target'] - 1

lmf = smf.logit('target ~ sep_wid + pet_wid', irisdf)
# setting alpha to 0 effectively removes the hyperparameter
results = lmf.fit_regularized(alpha=0)
print results.summary()
print np.exp(results.params)

# sklearn implementation
# note that by default, LogisticRegression() uses L2 Regularization.
# C, in this case, is the alpha parameter
# we can't remove it (it's a fickle sklearn library issue), but we can make is huge so it as less of an effect.
clf = lm.LogisticRegression(C=1e100).fit(irisdf[['sep_wid', 'pet_wid']], irisdf['target'])
print clf.coef_
print np.exp(clf.coef_)
# same model, different regularization method
clf2 = lm.LogisticRegression(penalty='l1', C=1e100).fit(irisdf[['sep_wid', 'pet_wid']], irisdf['target'])
print clf2.coef_
print np.exp(clf2.coef_)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.13699530933
            Iterations: 30
            Function evaluations: 30
            Gradient evaluations: 30
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  100
Model:                          Logit   Df Residuals:                       97
Method:                           MLE   Df Model:                            2
Date:                Sun, 24 Apr 2016   Pseudo R-squ.:                  0.8024
Time:                        18:16:06   Log-Likelihood:                -13.700
converged:                       True   LL-Null:                       -69.315
                                        LLR p-value:                 7.025e-25
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Inte